In [ ]:
def ocr_snf(passport):
    
    #gereftan ax scan shode
    #orgiganl_image=cv2.imread("images/passport2.PNG",cv2.IMREAD_COLOR)
    #bordan dar fazaie greyscale
    orgiganl_image=cv2.cvtColor(passport, cv2.COLOR_BGR2GRAY)
    #check quality if it is lower than 30000 reject it
    try:
        
        pre.check_iamge_quality(orgiganl_image,20000)
    except ValueError as e:
        sys.exit(str(e))

    #barabar akrdan DPI ha
    orgiganl_image=pre.equalize_dpi(orgiganl_image)
    #gereftan copy az ax asli
    copy_image=orgiganl_image.copy()
    #reverse krdn ax asli
    reversed_img=pre.revrese(copy_image)
    #thereshhold ax asli
    thresholded_img=pre.thresholding(reversed_img,120)
    #normal krdn ax asli
    normaled_img=pre.normalize(thresholded_img)
    image=normaled_img*255
    image = image.astype(np.uint8)
    #gereftan window ha va rikhtan toie araye
    windows_dict=getfiles.read_images_from_folder("windows")
    for key in windows_dict:

        window=windows_dict[key]
        window=pre.revrese(window)
        window=pre.thresholding(window,120)
        window=pre.normalize(window)
        window=window*255
        window = window.astype(np.uint8)
        ##########################################################################
        #gerftan jolo fildi k mikhaim
        pt1,pt2=basic.templatematching(image,window)
        #extract joloie on fild
        rectangled_image = normaled_img[pt1[1]:pt2[1], pt1[0]:pt2[0]]
        #remove kardan noise rectangled
        rectangled_image=rectangled_image*255
        rectangled_image=rectangled_image.astype(np.uint8)
        filtered_image = pre.remove_noise(rectangled_image,20)
        filtered_image=pre.normalize(filtered_image)
        #joda krdn charectare rectangled image
        chareters_image,_,_,_=extract.extract_charecter(filtered_image)

        # for i in range(len(chareters_image)):
        #     cv2.imshow("l",chareters_image[i])
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()
        #gereftan template charecter ha b soorat kham
        raw_char_template=getfiles.get_char_templates()
        #tele kardan on raw template ha va greyo reverese kardaneshon
        tele_templates=[]
        for i in range(len(raw_char_template)):
            tele_templates.append(basic.crop_templates(raw_char_template[i],0))
        #tele kardan template haie charecterha
        tele_char=[]
        for i in range(len(chareters_image)):
            tele_char.append(basic.crop_templates(chareters_image[i],1))
        #door rikhtan on haii az telechar ha ke masahateshon paiin tr az 50 bashe
        index_should_remove=[]
        for i in range(len(tele_char)):
            bool=sub.check_size_of_area(tele_char[i],30)
            if bool:
                pass
            else:
                index_should_remove.append(i)
        index_should_remove=np.sort(index_should_remove)  ##braie inke vaghty pop mikonim az bala pop konim k b moshkel nakhore
        index_should_remove =index_should_remove[::-1]    
        for index in index_should_remove:
            tele_char.pop(index)        

        #resize kardan template ha va albte emal threshhold 0.5
        resized_templates=[]
        for i in range(len(tele_templates)):
            hieght,width=tele_templates[i].shape
            target_width=width*(100/hieght)
            temp=pre.binary_threshold(pre.resize(tele_templates[i],100,int(target_width)),0.5)
            resized_templates.append(temp)
            
        #resized kardan charha bade tele kardan va hminotr emal threshold
        resized_char=[]
        for i in range(len(tele_char)):
            hieght,width=tele_char[i].shape
            target_width=width*(100/hieght)
            temp=pre.binary_threshold(pre.resize(tele_char[i],100,int(target_width)),0.5)
            resized_char.append(temp)

        #padd krrdan template ha bade resize shodan ba 10 ta pixel va normal kardaneshon zira chon bade padd krdn mishan unit 8 va 1 ha mishan 255
        padded_resize_template=[]
        for i in range(len(resized_templates)):
            h,w=resized_templates[i].shape
            #resized_templates[i]=resized_templates[i].astype(np.float32)
            temp=pre.normalize(pre.padding(resized_templates[i],h+10,w+10))
            padded_resize_template.append(temp)
        ################################################################################################3
        char_template_dataset=padded_resize_template
        ##################################################################################################
    
        #gereftan size maximum template ha bade padd kardan
        # max_width=0
        # max_hight=0

        # for i in range(len(padded_resize_template)):
        #     if padded_resize_template[i].shape[0]>max_hight:
        #         max_hight=padded_resize_template[i].shape[0]

        #     if padded_resize_template[i].shape[1]>max_width:
        #         max_width=padded_resize_template[i].shape[1]    

        #padd krdn char ha bade resize kardan b size hadeaksar templateha b alave 20 va normalsazi doabre chon mibre roe 255, 1 haro
        padded_resize_char=[]
        for i in range(len(resized_char)):
            temp=pre.normalize(pre.padding(resized_char[i],110+20,139+20))
            padded_resize_char.append(temp)  
        
        array_of_matched,help=basic.find_char(padded_resize_char,padded_resize_template)
        result_string=""
        for i in range(len(array_of_matched)):                                  
            if array_of_matched[i][3]==0:
                result_string=result_string+"A"
            elif array_of_matched[i][3]==1:
                result_string=result_string+"B"
            elif array_of_matched[i][3]==2:
                result_string=result_string+"C"
            elif array_of_matched[i][3]==3:
                result_string=result_string+"D"
            elif array_of_matched[i][3]==4:
                result_string=result_string+"E"    
            elif array_of_matched[i][3]==5:
                result_string=result_string+"F"
            elif array_of_matched[i][3]==6:
                result_string=result_string+"G"
            elif array_of_matched[i][3]==7:
                result_string=result_string+"H"
            elif array_of_matched[i][3]==8:
                result_string=result_string+"I"
            elif array_of_matched[i][3]==9:
                result_string=result_string+"J" 
            elif array_of_matched[i][3]==10:
                result_string=result_string+"K" 
            elif array_of_matched[i][3]==11:
                result_string=result_string+"L"
            elif array_of_matched[i][3]==12:
                result_string=result_string+"N"  
            elif array_of_matched[i][3]==13:
                result_string=result_string+"M" 
            elif array_of_matched[i][3]==14:
                result_string=result_string+"O" 
            elif array_of_matched[i][3]==15:
                result_string=result_string+"P" 
            elif array_of_matched[i][3]==16:
                result_string=result_string+"Q"
            elif array_of_matched[i][3]==17:
                result_string=result_string+"R"
            elif array_of_matched[i][3]==18:
                result_string=result_string+"S"
            elif array_of_matched[i][3]==19:
                check=post.check_I_T(array_of_matched[i][0])
                if check:
                    result_string=result_string+"T"
                else:
                    result_string=result_string+"I"
                        
            elif array_of_matched[i][3]==20:
                result_string=result_string+"U"
            elif array_of_matched[i][3]==21:
                result_string=result_string+"V"
            elif array_of_matched[i][3]==22:
                result_string=result_string+"W"
            elif array_of_matched[i][3]==23:
                result_string=result_string+"X"
            elif array_of_matched[i][3]==24:
                result_string=result_string+"Y"
            elif array_of_matched[i][3]==25:
                result_string=result_string+"Z"  

        head, sep, tail = key.partition('.')
        print(head+": "+result_string+"\n")

    return char_template_dataset

# for i in range(len(array_of_matched)):                                                                       
#     char=array_of_matched[i][0]
#     template_matched=array_of_matched[i][2]
#     cv2.imshow("char",char)
#     cv2.waitKey(0)
#     cv2.imshow("template_matched",template_matched)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()  

In [ ]:
def ocr_tpsd(passport,char_template_dataset):
    orgiganl_image=cv2.cvtColor(passport, cv2.COLOR_BGR2GRAY)
    orgiganl_image=pre.equalize_dpi(orgiganl_image)
    #gereftan copy az ax asli
    copy_image=orgiganl_image.copy()
    #reverse krdn ax asli
    reversed_img=pre.revrese(copy_image)
    #thereshhold ax asli
    thresholded_img=pre.thresholding(reversed_img,120)
    #normal krdn ax asli
    normaled_img=pre.normalize(thresholded_img)
    h_sum=sub.horizontal_summation(normaled_img)
    h_thresh=pre.binary_thresh_array(h_sum,20)
    cut_index=[]
    derative=sub.deriv(h_thresh)
    extract.find_cut_for_lines(derative,cut_index)
    size=len(cut_index)
    cut_index=cut_index[size-4:size]
    hhh,www=normaled_img.shape
    line1=normaled_img[cut_index[0]-3:cut_index[1]+3,0:www]
    line2=normaled_img[cut_index[2]-3:cut_index[3]+3,0:www]
    rectangled_image1=line1*255
    rectangled_image2=line2*255
    rectangled_image1=rectangled_image1.astype(np.uint8)
    rectangled_image2=rectangled_image2.astype(np.uint8)
    
    filtered_image1 = pre.remove_noise(rectangled_image1,20)
    filtered_image2 = pre.remove_noise(rectangled_image2,20)
    filtered_image1=pre.normalize(filtered_image1)
    filtered_image2=pre.normalize(filtered_image2)
    chareters_image1,_,_,_=extract.extract_charecter(filtered_image1)
    chareters_image2,_,_,_=extract.extract_charecter(filtered_image2)
    ######################################################################
    # for i in range(len(chareters_image1)):
    #     cv2.imshow("jds",chareters_image1[i])
    #     cv2.waitKey(0)
        
    # cv2.destroyAllWindows()   
    # for i in range(len(chareters_image2)):
    #     cv2.imshow("jds",chareters_image2[i])
    #     cv2.waitKey(0)
        
    # cv2.destroyAllWindows()   
    result_string=""
    ################extract type of passport#################
    unknown_type=chareters_image1[0]
    unknown_type=basic.crop_templates(unknown_type,1)
    h_un_type,w_un_type=unknown_type.shape
    unknown_type=pre.binary_threshold(pre.resize(unknown_type,100,int((w_un_type*(100/h_un_type)))),0.5)
    unknown_type=pre.normalize(pre.padding(unknown_type,110+20,139+20))
    arr_unknown_type=[]
    arr_unknown_type.append(unknown_type) ##chon onvar toie tabe find char byd araye bedim vgrna dochar moshkel mishe
    char_matched,help=basic.find_char(arr_unknown_type,char_template_dataset)
    if char_matched[0][3]==15:
        result_string="Type :"+"P\n"
    else:
        result_string="Type :"+"None\n"    
    ################extract type of passport#################
    
    #raw_numbers_template=func.get_numbers_templates()

    # for i in range(len(raw_numbers_template)):
    #     cv2.imshow("kkk",raw_numbers_template[i])
    #     cv2.waitKey(0)
    # cv2.destroyAllWindows()    
    # tele_numbers_templates=[]
    # for i in range(len(raw_numbers_template)):
    #     tele_numbers_templates.append(func.crop_templates(raw_numbers_template[i],0))


    tele_char1=[]
    for i in range(len(chareters_image1)):
        tele_char1.append(crop_templates(chareters_image1[i],1)) 
    tele_char2=[]
    for i in range(len(chareters_image2)):
        tele_char2.append(crop_templates(chareters_image2[i],1))  

    index_should_remove1=[]
    for i in range(len(tele_char1)):
        bool=func.check_size_of_area(tele_char1[i],30)
        if bool:
            pass
        else:
            index_should_remove1.append(i)
    index_should_remove1=np.sort(index_should_remove1)  ##braie inke vaghty pop mikonim az bala pop konim k b moshkel nakhore
    index_should_remove1 =index_should_remove1[::-1]    
    for index in index_should_remove1:
        tele_char1.pop(index)  

    index_should_remove2=[]
    for i in range(len(tele_char2)):
        bool=func.check_size_of_area(tele_char2[i],30)
        if bool:
            pass
        else:
            index_should_remove2.append(i)
    index_should_remove2=np.sort(index_should_remove2)  ##braie inke vaghty pop mikonim az bala pop konim k b moshkel nakhore
    index_should_remove2 =index_should_remove2[::-1]    
    for index in index_should_remove2:
        tele_char2.pop(index)

    # resized_numbers_templates=[]
    # for i in range(len(tele_numbers_templates)):
    #     hieght,width=tele_numbers_templates[i].shape
    #     target_width=width*(100/hieght)
    #     temp=func.binary_threshold(func.resize(tele_numbers_templates[i],100,int(target_width)),0.5)
    #     resized_numbers_templates.append(temp)

    resized_char1=[]
    for i in range(len(tele_char1)):
        hieght,width=tele_char1[i].shape
        target_width=width*(100/hieght)
        temp=func.binary_threshold(func.resize(tele_char1[i],100,int(target_width)),0.5)
        resized_char1.append(temp) 

    resized_char2=[]
    for i in range(len(tele_char2)):
        hieght,width=tele_char2[i].shape
        target_width=width*(100/hieght)
        temp=func.binary_threshold(func.resize(tele_char2[i],100,int(target_width)),0.5)
        resized_char2.append(temp) 

    # padded_resize_numbers_template=[]
    # for i in range(len(resized_numbers_templates)):
    #     h,w=resized_numbers_templates[i].shape
    #     #resized_templates[i]=resized_templates[i].astype(np.float32)
    #     temp=func.normalize(func.padding(resized_numbers_templates[i],h+10,w+10))
    #     padded_resize_numbers_template.append(temp)  

    # max_widthh=0
    # max_hightt=0

    # for i in range(len(padded_resize_numbers_template)):
    #     if padded_resize_numbers_template[i].shape[0]>max_hightt:
    #         max_hightt=padded_resize_numbers_template[i].shape[0]

    #     if padded_resize_numbers_template[i].shape[1]>max_widthh:
    #         max_widthh=padded_resize_numbers_template[i].shape[1]  

    padded_resize_numbers1=[]
    for i in range(len(resized_char1)):
        h,w=resized_char1[i].shape
        diff_with_h=max_hight-h
        diff_with_w=max_width-w
        temp=func.normalize(func.padding(resized_char1[i],h+diff_with_h+20,w+diff_with_w+20))
        padded_resize_numbers1.append(temp) 
    padded_resize_numbers2=[]
    for i in range(len(resized_char2)):
        h,w=resized_char2[i].shape
        diff_with_h=max_hight-h
        diff_with_w=max_width-w
        temp=func.normalize(func.padding(resized_char2[i],h+diff_with_h+20,w+diff_with_w+20))
        padded_resize_numbers2.append(temp)  

    array_of_matched1,help1=find_char(padded_resize_numbers1,padded_resize_template)
    array_of_matched2,help2=find_char(padded_resize_numbers2,padded_resize_template)  

    for i in range(len(array_of_matched1)):                                                                       
        char=array_of_matched1[i][0]
        template_matched=array_of_matched1[i][2]
        cv2.imshow("char",char)
        cv2.waitKey(0)
        cv2.imshow("template_matched",template_matched)
        cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [ ]:
known_values_dict = {}
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
for i, letter in enumerate(letters):
    known_values_dict[letter] = after_convert_template[i]

unknown_S=aftre_convert_char[0]
unknown_E=aftre_convert_char[1]
unknown_Y=aftre_convert_char[2]
unknown_E2=aftre_convert_char[3]
unknown_D=aftre_convert_char[4]
unknown_N=aftre_convert_char[5]
unknown_I=aftre_convert_char[6]
unknown_M=aftre_convert_char[7]
unknown_A=aftre_convert_char[8]

values_of_known=[]

for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    temp=[]
    for row in range(known_values_dict[char].shape[0]):
        for column in range(known_values_dict[char].shape[1]):
            value=known_values_dict[char][row,column]
            temp.append(str(row)+" "+str(column)+":"+str(value))

    values_of_known.append(temp) 


value_of_unknown_S=[]
value_of_unknown_E=[]
value_of_unknown_Y=[]
value_of_unknown_E2=[]
value_of_unknown_D=[]
value_of_unknown_N=[]
value_of_unknown_I=[]
value_of_unknown_M=[]
value_of_unknown_A=[]    


for row in range(unknown_A.shape[0]):
    for column in range(unknown_A.shape[1]):
        value=unknown_A[row,column]
        value_of_unknown_A.append(str(row)+" "+str(column)+":"+str(value))


with open("unknownA.txt", "w") as file:
    output_str = "\n".join(str(val) for val in value_of_unknown_A)
    file.write(output_str)

In [ ]:
rec_height,rec_width=rectangled_image.shape

temp_image=rectangled_image.copy()
# for i in range(len(cut_index_arr)):

#      cv2.line(temp_image, (cut_index_arr[i], 0), (cut_index_arr[i], hh - 1), (255, 255, 0), 1)
image_of_character=[]
for i in range(0,len(cut_index_arr)):
    if(i%2==0):
        char = rectangled_image[1:rec_height,cut_index_arr[i]:cut_index_arr[i+1]]
        # cv2.imshow('character image',char)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows() 
        image_of_character.append(char)
        trasnformed_char=char*255
        filename = f"sample{i//2 + 1}.jpg"
        cv2.imwrite("fathers name charecter/"+filename, trasnformed_char)

       
cv2.imshow('orginal', rectangled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()  


# hh,ww=rectangled_image.shape
# for i in range(len(array_of_cut)):

#     #cv2.line(rectangled_image, (array_of_cut[i], 0), (array_of_cut[i], hh - 1), (255, 255, 0), 1)

    
# cv2.imshow('Image with Vertical Line', rectangled_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows() 

In [ ]:
char_template=cv2.imread("fathers name charecter/sample3.jpg",cv2.IMREAD_COLOR)
char_template_grey= cv2.cvtColor(char_template, cv2.COLOR_BGR2GRAY)
thresholded_char_template=thresholding(char_template_grey)




#char_template=char_template.astype(np.float32)
# reversed_char_template=revrese(char_template)
# thresholded_char_template=thresholding(reversed_char_template)
# normaled_char_template=normalize(thresholded_char_template)


padded_charecter=[]
for i in range(len(image_of_character)):
      padded_charecter.append(padding(image_of_character[i],60,60))


    

# cv2.imshow('test1',image_of_character[0] )
     

best_match_val = None
best_match_char = None
for i in range(len(padded_charecter)):
    new_result=cv2.matchTemplate(padded_charecter[i],thresholded_char_template,cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(new_result)
    if best_match_val is None or max_val > best_match_val:
            best_match_val = max_val
            best_match_char=i


cv2.imshow('answer', padded_charecter[best_match_char])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)                # the first figure
plt.subplot(211)             # the first subplot in the first figure
plt.plot([1, 2, 3])
plt.subplot(212)             # the second subplot in the first figure
plt.plot([4, 5, 6])
plt.show()
# plt.figure(2)                # a second figure
# plt.plot([4, 5, 6])          # creates a subplot(111) by default

# plt.figure(1)                # figure 1 current; subplot(212) still current
# plt.subplot(211)             # make subplot(211) in figure1 current
# plt.title('Easy as 1, 2, 3') # subplot 211 title


# x = np.arange(len(sum_arr))
# plt.plot(x, sum_arr)
# plt.xlim(0, 200)
# plt.xlabel('Index')
# plt.ylabel('Value')
# plt.title('plot')
# plt.show()
# x = np.arange(len(copy_array))
# plt.plot(x, copy_array)
# plt.xlim(0, 200)
# plt.xlabel('Index')
# plt.ylabel('Value')
# plt.title('plot')
# plt.show()